In [ ]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split
from sklearn.base import clone
import timeit
import math

In [ ]:
def pca(F, X):
    n, d = X.shape
    mu = np.zeros((d, 1))
    Z = np.zeros((d, F))
    for i in range(d):
        mu[i] = (1. / n) * np.sum(X[:, [i]])
    X = X - mu.T
    U, s, Vt = la.svd(X, False)
    g = s[:F]
    for i in range(F):
        g[i] = 1. / g[i]
    W = Vt[:F]
    Z = np.dot(W.T, np.diag(g))
    return (mu, Z)

def pca_proj(X,mu,Z):
    n, d = X.shape
    X = X - mu.T
    return np.dot(X, Z)

In [ ]:
def k_fold(k, model, X, y):
    n, d = X.shape
    z = np.zeros((k, 1))
    for i in range(k):
        T = list(range(int((i * n) / k), int((n * (i + 1) / k))))
        S = [j for j in range(n) if j not in T]
        curr_model = clone(model)
        curr_model.fit(X[S], y[S])
        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(X[T])) ** 2)
    return z

In [ ]:
def bootstrapping(B, model, X, y):
    n, d = X.shape
    z = np.zeros((B, 1))
    for i in range(B):
        u = np.random.choice(n, n, replace=True)
        S = np.unique(u)
        T = np.setdiff1d(np.arange(n), S, assume_unique=True)
        curr_model = clone(model)
        curr_model.fit(X[u], y[u])
        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        # theta_hat will be d by 1
        z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(X[T])) ** 2)
    return z

In [ ]:
def evaluateModel(model, X, y, k=5, B=5):
    ########################KFOLD###################
    print('Evaluating K-fold with %d folds.' % k)
    start_time = timeit.default_timer()
    k_fold_z = k_fold(k, model, X, y)
    elapsed = timeit.default_timer() - start_time
    
    k_fold_mse = np.mean(k_fold_z)
    print('K-fold Mean Squared Error: ', k_fold_mse)
    
    k_fold_rmse = math.sqrt(k_fold_mse)
    print('K-fold Square Root Mean Squared Error: ', k_fold_rmse)

    print("Time elapsed for k-fold: ", elapsed)
    
    print()
    print()
    ###################BOOTSTRAPPING################
    print('Evaluating bootstrapping with %d bootstraps.' % B)
    start_time = timeit.default_timer()
    bootstrapping_z = bootstrapping(B, model, X, y)
    elapsed = timeit.default_timer() - start_time
    
    bootstrapping_mse = np.mean(bootstrapping_z)
    print('Bootstrapping Mean Squared Error: ', bootstrapping_mse)
    
    bootstrapping_rmse = math.sqrt(bootstrapping_mse)
    print('Bootstrapping Square Root Mean Squared Error: ', bootstrapping_rmse)

    print("Time elapsed for bootstrapping: ", elapsed)
    
    return (k_fold_z, bootstrapping_z)

# Data Processing

In [ ]:
data = pd.read_csv("train.csv", header=0)
print(data.shape)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]

print(X.shape)
print(Y.shape)

(1460, 81)
(1460, 80)
(1460, 1)


In [ ]:
# this just sums up how many nulls per feature and divides to find percentage of nulls per feature
# if over 50% null then print the feature
data_keys = X.keys()
for i, b in enumerate((X.isnull().sum() / X.shape[0]) > 0.5):
    if b:
        print(data_keys[i])

Alley
PoolQC
Fence
MiscFeature


In [ ]:
# data = data.drop(['Alley', 'MiscFeature', 'Fence', 'PoolQC'], axis=1)

In [ ]:
# Replaces categorical value in Quality columns with numerical scale
qualityCols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
              'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']

data[qualityCols].head()

for col in qualityCols:
    # NA is never used since all NA's got converted to NaN objects when pandas read in the csv
    data[col] = data[col].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA': 0})

data[qualityCols].head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond
0,4,3,4.0,3.0,5,4,NaN,3.0,3.0
1,3,3,4.0,3.0,5,3,3.0,3.0,3.0
2,4,3,4.0,3.0,5,4,3.0,3.0,3.0
3,3,3,3.0,4.0,4,4,4.0,3.0,3.0
4,4,3,4.0,3.0,5,4,3.0,3.0,3.0


In [ ]:
# categorical columns
catCols = set(list(X))-set(list(X._get_numeric_data()))
print(catCols)

# #TRY dropping all cat cols
# data = data.drop(columns=catCols)

{'MiscFeature', 'Street', 'HeatingQC', 'Condition2', 'ExterQual', 'GarageFinish', 'Foundation', 'ExterCond', 'Condition1', 'CentralAir', 'GarageType', 'PoolQC', 'Electrical', 'Neighborhood', 'BldgType', 'SaleType', 'Heating', 'RoofMatl', 'BsmtCond', 'LandSlope', 'Fence', 'BsmtQual', 'GarageQual', 'GarageCond', 'Functional', 'Alley', 'KitchenQual', 'MasVnrType', 'LotShape', 'FireplaceQu', 'BsmtFinType2', 'HouseStyle', 'SaleCondition', 'Exterior1st', 'LotConfig', 'BsmtExposure', 'RoofStyle', 'Utilities', 'PavedDrive', 'MSZoning', 'Exterior2nd', 'BsmtFinType1', 'LandContour'}


In [ ]:
#Perform one hot encoding on all categorical columns
frames = []
for col in catCols:
    oneHot_encoded = pd.get_dummies(X[col])
    oneHot_encoded = oneHot_encoded.add_prefix(col + '_is_')
    frames.append(oneHot_encoded)

X = X.drop(catCols, axis=1)

X = pd.concat(frames, axis=1)

In [ ]:
X.keys()

Index(['MiscFeature_is_Gar2', 'MiscFeature_is_Othr', 'MiscFeature_is_Shed',
       'MiscFeature_is_TenC', 'Street_is_Grvl', 'Street_is_Pave',
       'HeatingQC_is_Ex', 'HeatingQC_is_Fa', 'HeatingQC_is_Gd',
       'HeatingQC_is_Po',
       ...
       'BsmtFinType1_is_ALQ', 'BsmtFinType1_is_BLQ', 'BsmtFinType1_is_GLQ',
       'BsmtFinType1_is_LwQ', 'BsmtFinType1_is_Rec', 'BsmtFinType1_is_Unf',
       'LandContour_is_Bnk', 'LandContour_is_HLS', 'LandContour_is_Low',
       'LandContour_is_Lvl'],
      dtype='object', length=252)

In [ ]:
X.isnull().values.any()

False

In [ ]:
# 80:20 train test ratio
test_size = 0.2
# This function splits the training and target sets into random train and test subsets.
# X_train and X_test are subsets of the training data
# y_train and y_test are subsets the the target data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

# PCA Feature Selection

In [ ]:
F = 50

In [ ]:
X_mu, X_Z = pca(F, X.values)
X_pca = pca_proj(X.values, X_mu, X_Z)

In [ ]:
print(X_mu.shape)
print(X_Z.shape)
print(X_pca.shape)

(252, 1)
(252, 50)
(1460, 50)


In [ ]:
X_train_mu, X_train_Z = pca(F, X_train.values)

In [ ]:
print(X_train_mu.shape)
print(X_train_Z.shape)

(252, 1)
(252, 50)


In [ ]:
X_train_pca = pca_proj(X_train.values, X_train_mu, X_train_Z)
print(X_train_pca.shape)

X_test_pca = pca_proj(X_test.values, X_train_mu, X_train_Z)
print(X_test_pca.shape)

(1168, 50)
(292, 50)


# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor()
evaluateModel(adaBoost, X_pca, Y.values.ravel(), k=5, B=5)

adaBoost.fit(X_train_pca, y_train.values.ravel())
adaBoost.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  2967485092.57
K-fold Square Root Mean Squared Error:  54474.62797094985
Time elapsed for k-fold:  3.5247528999999993


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  3072207784.35
Bootstrapping Square Root Mean Squared Error:  55427.50025347117
Time elapsed for bootstrapping:  2.8703348999999996


0.61817786147401532

In [ ]:
#View Predicted values
predicted = adaBoost.predict(X_test_pca)
ada_pred = y_test.copy()
ada_pred['predicted'] = predicted
ada_pred.head()

,SalePrice,predicted
139,231500,236256.260700
865,148500,138743.586957
906,255000,249959.061856
42,144000,184720.612403
387,125000,143008.739264


# XGBoost Regressor

In [ ]:
#!pip3 install xgboost

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)

evaluateModel(xgb, X_pca, Y.values.ravel(), k=5, B=5)

xgb.fit(X_train_pca, y_train)
xgb.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  1966526292.19
K-fold Square Root Mean Squared Error:  44345.532945166044
Time elapsed for k-fold:  1.4109265000000022


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  1980887018.31
Bootstrapping Square Root Mean Squared Error:  44507.156933582766
Time elapsed for bootstrapping:  1.5316316000000008


0.7156087041545659

In [ ]:
predicted = xgb.predict(X_test_pca)
xgb_pred = y_test.copy()
xgb_pred['predicted'] = predicted
xgb_pred.head()

,SalePrice,predicted
139,231500,199052.828125
865,148500,118101.804688
906,255000,225448.937500
42,144000,174043.375000
387,125000,143039.562500


# SVM (SVR)

In [ ]:
from sklearn import svm

svr_model = svm.SVR(kernel="poly", coef0=-3500, gamma="auto")
# coef0 only works with poly and sigmoid kernels
# it just puts that value instead of the column of 1's

# without it, this model breaks for some reason

evaluateModel(svr_model, X_pca, Y.values.ravel(), k=5, B=5)

# epsilon, degree
svr_model.fit(X_train_pca, y_train.values.ravel())
svr_model.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  2128188088.28
K-fold Square Root Mean Squared Error:  46132.288998933785
Time elapsed for k-fold:  0.8688964999999982


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  2381540194.74
Bootstrapping Square Root Mean Squared Error:  48801.02657463045
Time elapsed for bootstrapping:  1.127999899999999


0.70917026967718344

In [ ]:
svr_predicted = svr_model.predict(X_test_pca)
svr_pred = y_test.copy()
svr_pred["predicted"] = svr_predicted
svr_pred.head()

,SalePrice,predicted
139,231500,194855.237660
865,148500,119792.734185
906,255000,208433.877896
42,144000,152581.316552
387,125000,137380.130075
